In [1]:
from pathlib import Path
import pandas as pd
from inference_gpt import generate_gpt_report
import os

processed_data_path = Path(os.getcwd()) / "output" / "cleaned"
unprocessed_data_path = Path(os.getcwd()) / "output"
event = "event2_new_prompt_fast"

In [2]:
df = pd.read_json(processed_data_path / f"{event}.json")
df.head(2)

,id,title,author,summary,body,timestamp,political_issue,is_about_politics,stakeholder,position,stance,sentiment,emotion,context,old_targets,country,support.type,support.content,targets,ISO-3
0,134245406231,Zelenskyy Hopes to Make Ukraine a Leading Weap...,Journalist of www.canadastandard.com,Ukrainian President Volodymyr Zelenskyy hosted...,by partnering with Western manufacturers so th...,2023-09-30 23:59:38,Military production and defense partnerships,True,vladimir zelensky,I hope to turn Ukraine into a military hub and...,Pro,Positive,Joy,Political,"[{'predicate': 'partner with', 'target': 'west...",ukraine,Argument,Zelenskyy described the first Defense Industri...,"[{'predicate': 'partner with', 'target': 'west...",UKR
1,134245406231,Zelenskyy Hopes to Make Ukraine a Leading Weap...,Journalist of www.canadastandard.com,Ukrainian President Volodymyr Zelenskyy hosted...,by partnering with Western manufacturers so th...,2023-09-30 23:59:38,Military production and defense partnerships,True,vladimir putin,The local elections in four Russian-annexed Uk...,Con,Negative,Disgust,Political,"[{'predicate': 'justify', 'target': 'russia's ...",russia,Opinion,Putin claimed that the residents chose to be p...,"[{'predicate': 'justify', 'target': 'russian a...",RUS


In [3]:
df[df["position"] == "I have received 73.38% of the votes in the elections."]

,id,title,author,summary,body,timestamp,political_issue,is_about_politics,stakeholder,position,stance,sentiment,emotion,context,old_targets,country,support.type,support.content,targets,ISO-3


In [4]:
# Define Catppuccin Mocha theme colors
catppuccin = {
    "base": "#303446",  # Background
    "surface": "#414559",  # Surface elements
    "text": "#c6d0f5",  # Text color
    "overlay0": "#737994",  # Node and edge color
    "overlay1": "#838ba7",
    "highlight": "#babbf1",  # Highlight (hover or accent)
    "accent": "#fab387",  # Edge labels/predicates
    "green": "#a6d189",  # Positive sentiment
    "blue": "#8caaee",  # Neutral sentiment
    "red": "#e78284",  # Negative sentiment
    "rosewater": "#f2d5cf",
    "subtext1": "#b5bfe2",
    "subtext0": "#a5adce",
    "teal": "#81c8be",
    "flamingo": "#eebebe",
    "mauve": "#ca9ee6",
    "lavender": "#babbf1",
    "sky": "#99d1db",
    "peach": "#ef9f76"
}

In [8]:
catppuccin_colorscale = [
    [0.0, catppuccin["teal"]],
    [0.1, catppuccin["sky"]],
    [0.2, catppuccin["blue"]],
    [0.3, catppuccin["rosewater"]],
    [0.7, catppuccin["red"]],
    [1.0, catppuccin["peach"]],
]

In [19]:
import plotly.graph_objects as go
import pandas as pd
from collections import Counter
import pycountry

def create_country_heatmap(df, row_numbers):
    """
    Generate a world heatmap that shows the number of positions mentioned per country.
    On hover, show the top 5 most popular positions.
    """
    # Filter the dataframe for the specified row numbers
    filtered_df = df.iloc[row_numbers]

    # Step 1: Aggregate positions by country
    country_positions = {}
    country_stakeholders = {}

    for _, row in filtered_df.iterrows():
        country = row['ISO-3']
        stakeholder = row['stakeholder']
        position = row['position']

        if country not in country_positions:
            country_positions[country] = []
            country_stakeholders[country] = set()  # Unique stakeholders

        country_positions[country].append(position)
        country_stakeholders[country].add(stakeholder)  # Track unique stakeholders per country

    # Step 2: Count unique positions per country
    country_position_counts = {country: len(positions) for country, positions in country_positions.items()}

    # Step 3: Get the top 5 positions by popularity (unique stakeholders) for each country
    country_top_positions = {}
    
    for country, positions in country_positions.items():
        # Count how many times each position appears (ranking by stakeholder count)
        position_counter = Counter(positions)
        top_positions = position_counter.most_common(5)  # Get top 5 positions
        
        # Prepare text for hover information
        top_position_text = '<br>'.join([f"{pos}: {count}" for pos, count in top_positions])
        country_top_positions[country] = top_position_text

    # Step 4: Prepare data for Plotly choropleth
    countries = list(country_position_counts.keys())
    counts = list(country_position_counts.values())
    hover_texts = [f"<b>{country}</b><br>Positions Count: {count}<br><br>Top 5 Positions:<br>{country_top_positions[country]}" 
                   for country, count in country_position_counts.items()]

    # Step 5: Create the choropleth map
    fig = go.Figure(data=go.Choropleth(
        locations=countries,  # Country codes
        locationmode='ISO-3',  # You can use 'ISO-3' for ISO3 codes
        z=counts,  # Color countries by position counts
        text=hover_texts,  # Custom hover text showing top positions
        hoverinfo='text',
        autocolorscale=False,
        colorscale=catppuccin_colorscale,  # Color scale from yellow to red
        colorbar_title="Positions Count",
        zmin=min(counts),  # Set the minimum value for the color scale
        zmax=max(counts),  # Set the maximum value for the color scale
    ))

    # Step 6: Update the layout for aesthetics
    fig.update_layout(
        title='World Heatmap: Positions by Country',
        geo=dict(
            showframe=True,
            showcoastlines=True,
            projection_type='natural earth'
        ),
        paper_bgcolor=catppuccin['overlay0'],  # Background color
        plot_bgcolor=catppuccin['base'],  # Plot area color
    )
    # make figure bigger(by making the figure area bigger not the area outside the figure, just the map)
    fig.update_layout(
        autosize=False,
        width=1200,
        height=800,
    )
    fig.show()

# Example usage with row numbers
row_numbers = list(range(0, df.shape[0]))  # Example of selected rows
create_country_heatmap(df, row_numbers)


In [22]:
pycountry.countries.search_fuzzy('turkiye')[0].alpha_3

'TUR'

In [26]:
print(pycountry.countries.get(name='Turkey'))
print(pycountry.countries.get(name='turkey'))
print(pycountry.countries.get(name='Türkiye'))
print(pycountry.countries.get(name='Turkiye'))

None
None
Country(alpha_2='TR', alpha_3='TUR', flag='🇹🇷', name='Türkiye', numeric='792', official_name='Republic of Türkiye')
None
